In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 2048
SAMPLE_PERIOD = 6
STRIDE = 4

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows={
            'train': {
                1: ("2014-01-01", "2014-02-01")
            },
            'unseen_activations_of_seen_appliances': {
                1: ("2014-02-02", "2014-02-08")                
            },
            'unseen_appliances': {
                2: ("2013-06-01", "2013-06-07")
            }
        }
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 112 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [3]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [4]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH
)

In [5]:
#FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [6]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

True

In [7]:
fig, axes = plt.subplots(2, sharex=True)
seq.all_appliances.plot(ax=axes[0])
axes[1].plot(seq.input)
fig.suptitle(FOLD)
plt.show()

In [8]:
sample = source.get_batch(num_seq_per_batch=1024)

In [9]:
pipeline = DataPipeline(
    [source],
    num_seq_per_batch=64,
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

In [ ]:
nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

In [ ]:
disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

In [ ]:
disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [14]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': True,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'seq_length': 2048,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'uniform_prob_of_selecting_each_model': True},
  'input_processing': [],
  'num_seq_per_batch': 64,
  'target_processing': [{'DivideBy': {'divisor': 215.8604}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [15]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {'dense_layer_0': 100}
    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [16]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [26]:
from pymongo import MongoClient
client = MongoClient()
db = client.neuralnilm_experiments
experiment_collection = db.e3

In [27]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net,
    data_pipeline=pipeline,
    collection=experiment_collection,
    metrics=Metrics([10]),
    learning_rate=1E-4,
    repeat_callbacks=[
        (  10, Trainer.validate)
    ]
)

In [28]:
trainer.fit(50)

INFO:neuralnilm.trainer:Starting training for 50 iterations.
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.trainer:Stopped training. Completed 50 iterations.


In [29]:
trainer.iteration

50

In [40]:
import pymongo
list(experiment_collection.scores.train.find().sort('_id', pymongo.DESCENDING).limit(-5))

[{u'_id': 49, u'loss': 1.656889621750097, u'source_id': 0},
 {u'_id': 48, u'loss': 1.1105482858219409, u'source_id': 0},
 {u'_id': 47, u'loss': 8.6273396880651, u'source_id': 0},
 {u'_id': 46, u'loss': 1.6861925984208015, u'source_id': 0},
 {u'_id': 45, u'loss': 1.0894585393641198, u'source_id': 0}]

In [41]:
list(experiment_collection.scores.validation.find())

[{u'SyntheticAggregateSource': {u'train': {},
   u'unseen_activations_of_seen_appliances': {},
   u'unseen_appliances': {}},
  u'_id': 0},
 {u'SyntheticAggregateSource': {u'train': {},
   u'unseen_activations_of_seen_appliances': {},
   u'unseen_appliances': {}},
  u'_id': 10},
 {u'SyntheticAggregateSource': {u'train': {},
   u'unseen_activations_of_seen_appliances': {},
   u'unseen_appliances': {}},
  u'_id': 20},
 {u'SyntheticAggregateSource': {u'train': {},
   u'unseen_activations_of_seen_appliances': {},
   u'unseen_appliances': {}},
  u'_id': 30},
 {u'SyntheticAggregateSource': {u'train': {},
   u'unseen_activations_of_seen_appliances': {},
   u'unseen_appliances': {}},
  u'_id': 40}]